In [276]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, remain_only_tridiagonal_elements, coeff_canonical_orthogonalization, coeff_gram_schmidt
from pytenet.global_krylov_method import generate_Hamiltonian_with_occupation_number, generate_reduced_H_non_ortho, coeff_canonical_orthogonalization_using_W
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn

from scipy.linalg import blas


Load and initialize datas: 

no is number of spatial orbitals

L is number of spatial orbitals, L = no

We DON'T need t_spin, g_spin now, only 'original' datas are enough 

r_THC is THC rank

In [2]:
#load integrals
with h5py.File("../data_water/eri_water.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("../data_water/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

7
(7, 28)
(28, 28)
rl errV: 2.8386751875274264e-12
abs errV: 2.0615721155266396e-11
errt: 7.097049412242525e-13
errh: 2.585427402664151e-13
errht: 9.079449636842276e-14


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [ ]:
#load Hamiltonian generated by above coefficients
H_correct = scipy.io.mmread('../data_water/H_water_correct.mtx').tocsr()
H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
e, v = sparse.linalg.eigsh(H_correct_10e, which = 'SA', k = 20)
# e_ground = e[0]
# e_1st_ex = e[1]

e_ground = -84.92262983120877
e_1st_ex = -84.52780562388604
e_2nd_ex = -84.46820215626565
e_3rd_ex = -84.42486181064207
e_4th_ex = -84.42379006099287

/home/ge49cag/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [126]:
e

array([-84.92262983, -84.52780562, -84.52780562, -84.52780562,
       -84.46820216, -84.42486181, -84.42486181, -84.42486181,
       -84.42379006, -84.42379006, -84.42379006, -84.38666878,
       -84.34814264, -84.34814264, -84.34814264, -84.32849318,
       -84.24285775, -84.24285775, -84.24285775, -84.23133678])

To construct 'original mpo' of molecular Hamiltonian, currently we can use ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy), which consider the spins automatically, we don't need generating spin-version datas anymore. The result of ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy) is mpo whose physical dimension is 4.

In [4]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 58, 96, 96, 58, 16, 1]


Generate Hartree-Fock state. See function generate_single_state for further details.

In [5]:
HFS = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])

Generate THC-MPO by layers, using THC tensors. 
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.
The physical bond_dim is 4.

In [6]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

In [7]:
#excited states for different molecules could have different total spins, please try different spin sections.
#for water in sto-6g: 1st excited state has spin 1
HFS = generate_single_state(7, [3, 3, 3, 3, 3, 0, 0])
#HFS.orthonormalize('left')
#HFS.orthonormalize('right')


We generate a group of orthogonal Krylov vectors using THC-MPO, with bond dim 40 for Krylov vectors. The vectors are stored in the folder = 'foldername', thus you don't have to generate them again for next time use. 

In [8]:
N_Krylov = 80
psi_original = copy.deepcopy(HFS)
max_bond_Krylov = 80
#max_bond_Krylov = 75
trunc_tol = 0
foldername = f"../water_Krylov"
#Krylov vectors are included in data, you dont have to run generate it. ofc, you can -regenerate it to verify the algorithm using the following code:
#generate_krylov_space_in_disk(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_thc, foldername)

#it indicates that even though during the calculation the bond dims exceed 40, but we only need 37 for Krylov vectors.

Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [135]:
N_use = 60
# C = coeff_canonical_orthogonalization(N_use, foldername)
# vector_list = generate_re_ortho_space_with_coeff(N_use, C, foldername)
#vector_list = generate_re_ortho_space(N_use, foldername)
H_reduced_non_rotho = generate_reduced_H_non_ortho(N_use, foldername, mpo_ref)
W = get_W(N_use, foldername)
coeff = coeff_gram_schmidt(N_use, foldername)
#coeff = coeff_canonical_orthogonalization_using_W(W)
# coeff = get_S(W)
# coeff = np.array(coeff)
H_reduced = np.einsum('ik, kl, jl -> ij', coeff.conj(), H_reduced_non_rotho, coeff)
H_reduced = remain_only_tridiagonal_elements(H_reduced)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:213: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:112: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


After achieving Krylov vectors, we implement Lanczos algorithm using these Krylov vectors. In this notebook, all the expectation values are calculated using ED for convenience. Since calculating the expectation value (contracting the tensor network) doesn't bring new errors, it makes no difference whether we use tensor network or ED to calculate the matrix elements for reduced Hamiltonian.

In [136]:
N_list = [0]
mps_start = copy.deepcopy(psi_original)
#error_list = [np.vdot(mps_start.as_vector(), H_correct_10e@(mps_start.as_vector())) - e_ground]

In [137]:
for N in range(5, N_use+1, 5):
    N_list.append(N)
    H_part = H_reduced[:N, :N]
    e_rotate, v_rotate = np.linalg.eigh(H_part)

    temp = v_rotate[:,0]
    temp = coeff[:N, :N].transpose(1,0)@temp
    temp = H_reduced_non_rotho[:N, :N]@temp
    temp = coeff[:N, :N].conj()@temp
    e_rotate_ground = ((v_rotate[:,0].reshape(1, N)).conj())@temp
    #e_rotate_ground = np.einsum(', ,  -> ', v_rotate_ground_coeff.conj(), H_reduced_non_rotho, v_rotate_ground_coeff)
    #e_new = np.vdot(v_rotate_ground, H_correct_10e@v_rotate_ground)
    #error_list.append(e_rotate_ground - e_ground)
    
    print(e_rotate_ground - e_ground)
    #result gets worse when N_krylov increase, since the orthogonality gets worse.
    #One need to re-start the Lanczos to solve this issue.

[0.35244435]
[0.02484897]
[0.00195854]
[7.23079902e-05]
[8.51511103e-07]
[1.42243408e-08]
[1.02659214e-10]
[1.42108547e-14]
[-1.42108547e-13]
[-7.10542736e-14]
[1.42108547e-14]
[-1.27897692e-13]


In [185]:
N_use = 70

W = get_W(N_use, foldername)
#W = (W + W.T)/2
coeff = np.array(get_S(W))
#coeff = coeff_canonical_orthogonalization_using_W(W)
#coeff = coeff_gram_schmidt(N_use, foldername)

# is_real = np.isrealobj(coeff)
# print("is real:", is_real)
# for i in range(N_use):
#     for j in range(N_use):
#         if i!= j:
#             print(i, j, W[i, j] - W[j, i])

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:112: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [285]:
W = get_W(N_use, foldername)
#W = (W + W.T)/2
W = W.astype(np.float64)
W = get_W(N_use, foldername)

In [286]:


#_ , new_coeff = np.linalg.eigh(W)

#W_2 = np.einsum('ca, db, ab -> cd', new_coeff.T, new_coeff.T, W)


D, U = np.linalg.eigh(W) 
sqrt_D = np.sqrt(D)
inverse_sqrt_D = 1 / sqrt_D
D_invers_sq_root = np.diag(inverse_sqrt_D)
coeff= D_invers_sq_root@U.T

#new_coeff = U @ D_invers_sq_root @ U.T
#new_coeff = U @ D_invers_sq_root @ U.T
#new_coeff = D_invers_sq_root @ U.T

#W_2 = np.einsum('ca, db, ab -> cd', coeff, coeff, W)
#W_2 = coeff @ W @ coeff.T

W_temp = coeff @ W
W_2 = blas.dsyrk(alpha=1.0, a=W_temp, lower=0, trans=0)



#W_2 = np.einsum('ac,ab,bd->cd', coeff.T, W, coeff)
#W_2 = coeff.conj().T @ W @ coeff
#W_2 = (W_2 + W_2.T)/2 

for i in range(N_use):
    for j in range(N_use):
        if i != j:
            print(i, j, W_2[i, j] - W_2[j, i])

0 1 -2.102859618768904e-12
0 2 -4.6596189704139456e-12
0 3 -4.2487261116424245e-11
0 4 -2.75344737104937e-10
0 5 3.3126576143977627e-10
0 6 6.357837781565777e-10
0 7 1.42589748755919e-09
0 8 -1.110967751975916e-09
0 9 1.3615220885551783e-10
0 10 -1.5244653524369237e-09
0 11 -5.089777938069919e-10
0 12 -1.8360255971437337e-09
0 13 -3.2303497804256303e-10
0 14 1.3863514260010576e-10
0 15 -7.28371193092157e-10
0 16 -9.791794340636535e-10
0 17 -3.8412227556661765e-10
0 18 -1.434802476990318e-10
0 19 1.5575351536987576e-09
0 20 -1.0973604136557368e-11
0 21 4.680543397366444e-10
0 22 8.436850815434857e-10
0 23 1.3154213576409812e-09
0 24 -2.0232968382131983e-09
0 25 -7.861420180758044e-10
0 26 -3.783048131411588e-10
0 27 1.0535057416297483e-09
0 28 -5.865407253349656e-10
0 29 8.979248503075312e-10
0 30 2.37240497579411e-10
0 31 1.8551466711406231e-09
0 32 -1.025162345477049e-09
0 33 1.9224156667884796e-09
0 34 -4.6121036677606134e-10
0 35 -4.758630601389399e-10
0 36 1.86432173175567e-09
0 37

In [263]:
import numpy as np
from scipy.linalg import cholesky, solve_triangular

# Define your symmetric positive-definite matrix B
# For example, create a random positive-definite matrix
np.random.seed(0)
size = 70
A_random = np.random.rand(size, size)
B = copy.deepcopy(W_2)  # Ensures B is symmetric positive-definite

# Ensure B is symmetric
if not np.allclose(B, B.T):
    raise ValueError("Matrix B must be symmetric.")

# Perform the Cholesky decomposition of B
try:
    L = cholesky(B, lower=True, check_finite=True)
except np.linalg.LinAlgError:
    raise ValueError("Matrix B must be positive-definite for Cholesky decomposition.")

# Instead of computing L_inv, solve the linear systems
# Compute A such that A = inv(L)
identity_matrix = np.eye(size)
A = solve_triangular(L, identity_matrix, lower=True, check_finite=True)

# Verify that A B A^T equals the identity matrix
I_computed = A @ B @ A.T
I_actual = np.identity(B.shape[0])

# Calculate the maximum off-diagonal element in I_computed
off_diagonal_elements = I_computed - np.diag(np.diag(I_computed))
max_off_diagonal = np.max(np.abs(off_diagonal_elements))

print(f"Maximum off-diagonal element in A B Aᵗ: {max_off_diagonal:.2e}")

# Check if the computed identity matrix is close to the actual identity matrix
if np.allclose(I_computed, I_actual, atol=1e-8):
    print("A B Aᵗ equals the identity matrix within tolerance.")
else:
    print("A B Aᵗ does not equal the identity matrix within tolerance.")

# Output the maximum deviation from the identity matrix
max_deviation = np.max(np.abs(I_computed - I_actual))
print(f"Maximum deviation from identity: {max_deviation:.2e}")


Maximum off-diagonal element in A B Aᵗ: 1.91e-22
A B Aᵗ equals the identity matrix within tolerance.
Maximum deviation from identity: 2.22e-16


In [267]:
# W_2 = coeff @ W @ coeff.T
# #W_2 = np.einsum('ac,ab,bd->cd', coeff.T, W, coeff)
# #W_2 = coeff.conj().T @ W @ coeff
# W_2 = (W_2 + W_2.T)/2

# for i in range(N_use):
#     for j in range(N_use):
#         if i!= j:
#             print(i, j, W_2[i, j])

In [ ]:
#given a symmetric W_2, it could be converted to idenity
#W1 is ill
#And, since W1 is symmetric, W2 should also be symmetric, how to do it?
#can we just use non-ortho krylov? so that we could have a good starting W.

norm(A @ W_2 @ A.T - np.identity(70))

1.2947314098278137e-15

In [271]:
#coeff_2_indirect = np.array(coeff_canonical_orthogonalization_using_W(W_2))
#coeff_2_indirect = np.array(get_S(W_2))
coeff_2_indirect = copy.deepcopy(A)
coeff_2 = np.einsum('ki, ij -> kj', coeff_2_indirect, coeff)
#W_3 = np.einsum('ca, ab, db -> cd', coeff_2.conj(), W, coeff_2)
W_3 = coeff_2_indirect @ W_2 @ coeff_2_indirect.T
#W_3 = coeff_2 @ W @ coeff_2.T
#W_3 = (W_3 + W_3.T)/2

for i in range(N_use):
    for j in range(N_use):
        if i!= j:
            #print(i, j, W_3[i, j] - W_3[j, i])
            print(i, j, W_3[i, j])
        

0 1 -1.1787064837048134e-07
0 2 -6.410292406469457e-10
0 3 4.271636353953717e-09
0 4 -7.694834143209788e-10
0 5 7.589025705633519e-10
0 6 -1.6928939933463976e-10
0 7 -3.230312983083386e-11
0 8 1.4158425699950797e-10
0 9 -2.1562759739373703e-10
0 10 -3.295421450652452e-10
0 11 6.433182629753132e-11
0 12 7.126800164957345e-11
0 13 1.3502517749532681e-11
0 14 3.54637950073007e-10
0 15 -2.302034207997018e-12
0 16 -1.2791810051038496e-10
0 17 -1.6294089285698388e-11
0 18 5.0943054716996e-10
0 19 -5.542385266118789e-10
0 20 5.827009817609645e-10
0 21 2.3106525589442904e-10
0 22 5.209960043430293e-10
0 23 -1.9795730695469995e-10
0 24 4.382967913331674e-11
0 25 1.0451225060785613e-10
0 26 2.300673421435351e-11
0 27 1.301674775741256e-10
0 28 -4.545955333558249e-10
0 29 -5.6514318886093e-12
0 30 8.923306091300118e-11
0 31 -2.942294115673249e-10
0 32 8.15226123965762e-11
0 33 5.937413561395465e-12
0 34 2.3903456633582953e-11
0 35 9.436754697954555e-11
0 36 -2.1775102777915916e-10
0 37 3.38484919

In [226]:
coeff_2_indirect = np.array(get_S(W_2))
coeff_2 = np.einsum('ki, ij -> kj', coeff_2_indirect, coeff)
#W_3 = np.einsum('ca, ab, db -> cd', coeff_2.conj(), W, coeff_2)
W_3 = np.einsum('ca, db, ab -> cd', coeff_2, coeff_2, W)
W_3 = (W_3 + W_3.T)/2

for i in range(N_use):
    for j in range(N_use):
        if i!= j:
            #print(i, j, W_3[i, j] - W_3[j, i])
            print(i, j, W_3[i, j])
        

0 1 -1.9017491581507784e-08
0 2 5.846843009749136e-10
0 3 -1.8485756925201713e-08
0 4 -5.1782276244694e-09
0 5 -3.772635537302449e-10
0 6 -3.2846599051783443e-10
0 7 -3.134879023036774e-10
0 8 5.951248382984886e-10
0 9 1.5587531265737198e-11
0 10 -1.0657952298487317e-10
0 11 -5.618212561842029e-10
0 12 -1.3813852839383856e-10
0 13 1.2505774193982688e-10
0 14 -1.622020295855009e-10
0 15 -4.309394507906461e-11
0 16 -2.2558475920586574e-10
0 17 -2.7695530591120843e-10
0 18 -6.236749727772661e-10
0 19 3.2467271232627937e-10
0 20 -2.069335076557155e-10
0 21 -3.8717081971006984e-10
0 22 -4.5774042282259453e-10
0 23 6.749731884525545e-10
0 24 -8.346436458639417e-10
0 25 -2.457554992840727e-12
0 26 -1.7431197110728824e-10
0 27 -1.0973535101432841e-10
0 28 3.209518241453768e-10
0 29 -3.8176213492530087e-10
0 30 2.2198800957162756e-10
0 31 -1.0227462973744217e-10
0 32 6.045606532750769e-10
0 33 -6.769538696965727e-10
0 34 4.3501992199462824e-10
0 35 -4.689411445962799e-11
0 36 1.6341879412185012

In [228]:
coeff_3_indirect = np.array(get_S(W_3))
coeff_3 = np.einsum('ki, ij -> kj', coeff_3_indirect, coeff_2)
#W_3 = np.einsum('ca, ab, db -> cd', coeff_2.conj(), W, coeff_2)
W_4 = np.einsum('ca, db, ab -> cd', coeff_3, coeff_3, W)
W_4 = (W_4 + W_4.T)/2

for i in range(N_use):
    for j in range(N_use):
        if i!= j:
            #print(i, j, W_3[i, j] - W_3[j, i])
            print(i, j, W_4[i, j])
        

0 1 1.4817018723078945e-07
0 2 1.2586308884010577e-08
0 3 5.0565311937589286e-09
0 4 8.135478957171927e-10
0 5 -2.750963901121395e-10
0 6 2.878567696706913e-10
0 7 -2.2679635947042698e-10
0 8 -4.5813108862091667e-10
0 9 -4.873790260262467e-11
0 10 -2.4512475382820753e-10
0 11 4.067000070051563e-10
0 12 4.00302263559027e-10
0 13 7.582245942217014e-11
0 14 2.4548546528890824e-10
0 15 2.0989779359048555e-11
0 16 2.2574928038032738e-10
0 17 1.4761616234926223e-10
0 18 4.148993995589656e-10
0 19 4.18839962712525e-11
0 20 1.9709369119408349e-10
0 21 1.8573452498227283e-10
0 22 5.895430662747336e-10
0 23 -7.488170031960673e-10
0 24 1.2353505718376567e-10
0 25 -1.6214885684151525e-10
0 26 6.043210029615942e-11
0 27 4.8064453458951384e-11
0 28 2.137231190635358e-10
0 29 -1.8013936696481547e-11
0 30 4.434519921409086e-10
0 31 6.686688008850084e-11
0 32 -1.612095769376598e-10
0 33 2.8468717661045417e-10
0 34 -1.6686480409228155e-10
0 35 1.7442218069180226e-10
0 36 7.494147490072489e-11
0 37 1.228

In [230]:
coeff_4_indirect = np.array(get_S(W_4))
coeff_4 = np.einsum('ki, ij -> kj', coeff_4_indirect, coeff_3)
#W_3 = np.einsum('ca, ab, db -> cd', coeff_2.conj(), W, coeff_2)
W_5 = np.einsum('ca, db, ab -> cd', coeff_4, coeff_4, W)
#W_5 = (W_5 + W_5.T)/2

for i in range(N_use):
    for j in range(N_use):
        if i!= j:
            #print(i, j, W_4[i, j] - W_4[j, i])
            print(i, j, W_5[i, j])
        

0 1 1.2048204780512606e-06
0 2 -2.5630448874380818e-09
0 3 4.170231449052153e-08
0 4 -2.938356225001826e-09
0 5 1.5100223293984527e-09
0 6 -1.1328493698670172e-10
0 7 -4.446434331839555e-10
0 8 1.0295790886516443e-09
0 9 1.4434675676966435e-10
0 10 -6.688180009817302e-10
0 11 -6.816058828462701e-11
0 12 -4.198755232387441e-10
0 13 -2.3003488003325856e-10
0 14 1.8939794177441627e-10
0 15 1.1056244808571591e-10
0 16 -3.655441227490286e-10
0 17 -7.683664468571827e-11
0 18 6.39780423083991e-10
0 19 8.92625438841943e-10
0 20 3.316636982414467e-10
0 21 1.5014018500680848e-10
0 22 -4.2455297610821674e-10
0 23 -1.2350248601578073e-10
0 24 1.5599767311247348e-10
0 25 5.16307774311997e-10
0 26 -8.539186718836689e-11
0 27 -2.4131710171793053e-10
0 28 -3.210855956115033e-10
0 29 5.13188578438184e-10
0 30 8.446233296099948e-12
0 31 -2.937496391963723e-10
0 32 6.820300886556385e-10
0 33 -2.4560024153297277e-11
0 34 -1.6506095101842533e-11
0 35 -1.66174313892542e-10
0 36 5.65804156998917e-10
0 37 -7.

In [176]:
coeff_5_indirect = np.array(get_S(W_5))
coeff_5 = np.einsum('ki, ij -> kj', coeff_5_indirect, coeff_4)
#W_3 = np.einsum('ca, ab, db -> cd', coeff_2.conj(), W, coeff_2)
W_6 = np.einsum('ca, db, ab -> cd', coeff_5, coeff_5, W)
W_6 = (W_6 + W_6.T)/2

for i in range(N_use):
    for j in range(N_use):
        if i!= j:
            #print(i, j, W_4[i, j] - W_4[j, i])
            print(i, j, W_6[i, j])
        

0 1 0.0
0 2 0.0
0 3 0.0
0 4 0.0
0 5 0.0
0 6 0.0
0 7 0.0
0 8 0.0
0 9 0.0
0 10 0.0
0 11 0.0
0 12 0.0
0 13 0.0
0 14 0.0
0 15 0.0
0 16 0.0
0 17 0.0
0 18 0.0
0 19 0.0
0 20 0.0
0 21 0.0
0 22 0.0
0 23 0.0
0 24 0.0
0 25 0.0
0 26 0.0
0 27 0.0
0 28 0.0
0 29 0.0
0 30 0.0
0 31 0.0
0 32 0.0
0 33 0.0
0 34 0.0
0 35 0.0
0 36 0.0
0 37 0.0
0 38 2.6469779601696886e-23
0 39 0.0
0 40 0.0
0 41 0.0
0 42 0.0
0 43 6.776263578034403e-21
0 44 0.0
0 45 8.131516293641283e-20
0 46 -2.0328790734103208e-19
0 47 6.776263578034403e-21
0 48 -1.463672932855431e-18
0 49 -7.318364664277155e-19
0 50 -1.4907779871675686e-19
0 51 -1.463672932855431e-18
0 52 6.505213034913027e-19
0 53 4.336808689942018e-19
0 54 -1.734723475976807e-18
0 55 1.734723475976807e-18
0 56 -2.0816681711721685e-17
0 57 1.5612511283791264e-17
0 58 -5.204170427930421e-17
0 59 -4.163336342344337e-17
1 0 0.0
1 2 0.0
1 3 0.0
1 4 -2.524354896707238e-29
1 5 0.0
1 6 0.0
1 7 0.0
1 8 0.0
1 9 0.0
1 10 0.0
1 11 0.0
1 12 0.0
1 13 4.0389678347315804e-28
1 14 0.0
1 1

In [166]:
#coeff_6_indirect = np.array(get_S(W_6))
coeff_6_indirect = coeff_canonical_orthogonalization_using_W(W_6)

coeff_6 = np.einsum('ki, ij -> kj', coeff_6_indirect, coeff_5)
#W_3 = np.einsum('ca, ab, db -> cd', coeff_2.conj(), W, coeff_2)
W_7 = np.einsum('ca, db, ab -> cd', coeff_6, coeff_6, W)

for i in range(N_use):
    for j in range(N_use):
        if i!= j:
            #print(i, j, W_4[i, j] - W_4[j, i])
            print(i, j, W_7[i, j])
        

0 1 8.276876528998585e-30
0 2 -2.7467150643664105e-28
0 3 -2.4375801971329265e-28
0 4 -1.2448225084387566e-27
0 5 1.9721522630525295e-31
0 6 -2.1772560984099926e-28
0 7 -7.415292509077511e-29
0 8 -1.939020105033247e-27
0 9 8.519697776386927e-29
0 10 6.058451752097371e-28
0 11 -1.2243121249030103e-27
0 12 -2.527510340328122e-27
0 13 1.1296488162764889e-27
0 14 1.830551730565358e-27
0 15 2.8903863567297873e-27
0 16 6.124716068135936e-27
0 17 3.322682132790902e-27
0 18 8.801321119550829e-27
0 19 2.0268203237843456e-26
0 20 -7.860998920527383e-27
0 21 1.631344630474422e-26
0 22 8.947383646533157e-26
0 23 -1.2040186781161998e-26
0 24 -3.276021010247079e-26
0 25 1.2836736615018586e-25
0 26 -1.4148771305177337e-25
0 27 -1.5843097850767506e-25
0 28 -9.990175364248594e-25
0 29 -2.5000657153610873e-25
0 30 -2.424268856529126e-24
0 31 -1.2333770418910736e-25
0 32 -1.4419781634197137e-24
0 33 -4.300006373345196e-24
0 34 4.97312241730849e-24
0 35 1.2645033814048366e-23
0 36 -1.408160326134203e-23
0

In [ ]:
def gram_schmidt_based_on_W()